In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datascience

In [ ]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Cleaning the Data and Making the Tables

In [ ]:
masks = Table.read_table('../input/state-political-party-vs-maskvaccination-data/masks.csv')
vaccine_acceptance = Table.read_table('../input/state-political-party-vs-maskvaccination-data/vaccine_acceptance.csv')
masks

In [ ]:
vaccine_acceptance #this table also has the State's Political party which will be our target variable later on

## Renaming the columns in the Table

In [ ]:
masks = masks.with_column('%_wearing_masks', masks.column(1)).drop(1)
vaccine_acceptance = vaccine_acceptance.with_column('%_fine_with_vaccine', vaccine_acceptance.column(1)).drop(1)
masks

In [ ]:
vaccine_acceptance

In [ ]:
#reordering column in vaccine_acceptance table

vaccine_acceptance = vaccine_acceptance.with_column("Political_Party", vaccine_acceptance.column(1)).drop(1)
vaccine_acceptance

In [ ]:
#combining the tables

compiled_table = masks.with_columns(
    '%_fine_with_vaccine', vaccine_acceptance.column(1),
    'Political_Party', vaccine_acceptance.column(2)
)

compiled_table

In [ ]:
compiled_table_democrat = compiled_table.where('Political_Party', 'D')
compiled_table_republican = compiled_table.where('Political_Party', 'R')
compiled_table_democrat

# Graphing our Table Data

In [ ]:
x = compiled_table_democrat.column(1)
y = compiled_table_democrat.column(2)
plt.scatter(x, y, color = 'blue', label = "Democratic State")

x = compiled_table_republican.column(1)
y = compiled_table_republican.column(2)
plt.scatter(x, y, color = 'red', label = "Republican State")
plt.xlabel('% of population wearing Masks')
plt.ylabel('% of population comfortable with vaccine')

plt.legend()
plt.show()

# Machine Learning Portion

In [ ]:
# Now we just need to import and use scikit learn to create a model that predicis if states are republican or demographic based on mask and vaccine data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [ ]:
df = compiled_table.to_df()
features = ['%_wearing_masks', '%_fine_with_vaccine']
X = df[features]
y = df['Political_Party']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix